## corpus data clean and get keywords

In [106]:
# !pip install -i https://test.pypi.org/simple/ krovetz

In [107]:
# test data
import pandas as pd
df = pd.read_csv('./NABU FAQ for Worry Free Product 20190401.csv')
df.head(10)

,ID,A,ACCOUNTID,DESCRIPTION,CLOSEDDATE,ISESCALATED,CREATEDDATE,TS_PRODUCT_NAME__C,TS_PRODUCT_VERSION__C,TS_EMAIL_TEXTBODY__C,TS_CASE_SUPPORT_REGION__C,FAQID,Column1
0,5000B00000dG79dQAC,514391,001U000000POQggIAH,"Hi, we are a TM reseller and we have a custome...",2017-08-03 13:27:55,0,2017-07-26 18:33:24,Worry Free Services for Dell,Not Applicable,"Hi Kevin,\n\nI will now proceed in closing thi...",NABU,1102395;,Y
1,5000B00000gxIc8QAE,681829,001U0000016Y6SBIA0,My computer was encrypted while using your pro...,2018-02-22 14:48:55,0,2018-01-05 16:26:37,Worry Free Services for Dell,5.3,"Hi Dennis,\n\nI will now proceed in closing th...",NABU,1039099;,Y
2,5000B00000ZlwGfQAJ,309921,001U000000fwMAWIA2,AC: WF-3VB2-XLSDL-33VTA-6GZP8-A4ASM-8AJFN,2017-01-10 18:14:24,0,2017-01-08 13:53:53,Worry-Free Bs Services For Dell,NaN,I'm on the Trend Micro website and want to kno...,NABU,1038246;,Y
3,5000B00000Z9WUNQA3,306770,001U000000gAy3qIAC,[Customer Information]\nCustomer's Name: craig...,2017-01-02 18:32:05,0,2017-01-02 18:15:26,Worry-Free Business Security Advanced,NaN,"Hi Craig,\n\nThis would be the article for mig...",NABU,1037744;1060319;,N
4,5000B00000ZlimFQAR,309036,001U000000gxDh4IAE,This is in reference to an older ticket that I...,2017-02-01 14:02:16,0,2017-01-05 20:57:53,Worry-Free Business Security Advanced,9.0,"Dear Chris,\n\nUpon reviewing your Service Req...",NABU,1112119;,Y
5,5000B00000ZlpUmQAJ,309629,001U000000TPosUIAT,Name: Levi Swanson\nSession ID: 495099828\nSta...,2017-01-07 2:05:28,0,2017-01-06 18:10:11,Worry-Free Business Security Advanced,9,"Hi Levi,\n\nGood day! \n\nThis is Paolo from T...",NABU,1039099;,Y
6,5000B00000Z9ZmtQAF,307096,001U000000PPeRcIAL,[Environment]\n-Product: \t\t\t\t[WFBS-A]\n-OS...,2017-01-03 15:59:25,0,2017-01-03 14:33:30,Worry-Free Business Security Advanced,NaN,"Hi Dana,\n\nThis is the article regarding your...",NABU,1061328;,N
7,5000B00000ZlkSgQAJ,309114,001U000000fxWvVIAU,[Customer Information]\nCustomer's Name: lesli...,2017-01-06 0:46:39,0,2017-01-06 0:39:14,Worry-Free Business Security Advanced,NaN,"Hi Leslie,\n\nThis is the article for reconnec...",NABU,1037744;,Y
8,5000B00000ZmB7zQAF,311588,001U000000fwlJkIAI,When I attempt to install the client on a Wind...,2017-02-07 16:27:06,0,2017-01-10 17:20:55,Worry-Free Business Security Advanced,9.0,"Hi Ed,\n\nFor the meantime, we will proceed to...",NABU,1057653;,N
9,5000B00000Z9gr3QAB,307476,001U0000010RuBuIAK,Trend Micro Messaging Security Agent Master Se...,2017-01-05 15:14:12,0,2017-01-04 1:16:57,Worry-Free Business Security Advanced,9.0,"Patrick,\n\nLast night I updated our Exchange ...",NABU,1096694;1037599;,N


In [108]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

import krovetz
from nltk.tokenize import word_tokenize

stop_words=stopwords.words('english')
break_words=list('\n\t')
prefix_char_remove=list('.*,-')

stemmer = krovetz.PyKrovetzStemmer()
#stemmer = SnowballStemmer("english")
#stemmer.stem('working')
#lemmatizer = WordNetLemmatizer()
word_freqs = dict()


## 字詞清理函數

## Test
```
import krovetz
ks = krovetz.PyKrovetzStemmer()
ks.stem('Microsoft Edge'.lower()) # 使用 NLTK stemming 會傳回字根(會漏字母 e)，而非原形
ks.stem('working'.lower()) # 無法去除 ing
```

In [109]:
# not used
def stem_phrase(word_0):
    word_list=[]
    for word in word_0.split(' '):
        word_list.append(stemmer.stem(word))
        print(word, stemmer.stem(word))
    return ' '.join(word_list)

# clean words
def clean_word(word_0):
    word_list=[]
    for word in word_0.split('/'):

        if len(word) <= 0 :
            continue

        remote_path=word.split('\\')
        if len(remote_path) > 0 and len(remote_path[-1])>0:
            word=remote_path[-1]
        elif len(remote_path) > 1 and len(remote_path[-2])>0:
            word=remote_path[-2]
        elif len(remote_path) > 2 and len(remote_path[-3])>0:
            word=remote_path[-3]

        if word[0] == '"' and word[-1] == '"':
            word=word[1:-1]
        if word[0] == "'" and word[-1] == "'":
            word=word[1:-1]
        if len(word) <= 0 :
            continue

        # repeat 3 times to remove continous chars
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]

        # lemmatize
        word=word.strip().lower()
        word=stemmer.stem(word)
        if word=='':
            continue      
            
        # remove .exe and .com
        if word.endswith('.exe') or word.endswith('.com'):
            word=word[:-4]
            
        # remove the words with one or two characters only
        if len(word) <= 2 :
            continue
        if not (word in stop_words):
            word_list.append(word)
               
    return word_list

### test clean_word()
```
line = 'server\\pccsrv\\wss\\spsc\\bin aaa'
words = word_tokenize(line) #line.lower().split(' ')
for word_0 in words:        
    clean_word(word_0)
email_words=word_freqs.keys()            
print(email_words)
```

## 分詞

In [110]:
# 清理後的 email DESCRIPTION
clean_corpus=[]
original_corpus=[]
for index, line in df.iterrows():
    clean_line=""
    line = line['DESCRIPTION']
    for break_word in break_words:
        #print('-',len(break_word), break_word, '-')
        line = line.replace(break_word, ' ')
    words = word_tokenize(line) #line.lower().split(' ')
    for word_0 in words:        
        word_list = clean_word(word_0)
        for word in word_list:        
            if word in word_freqs:
                word_freqs[word] += 1
            else:
                word_freqs[word] = 1
            clean_line+=' '+word
    original_corpus.append(line)
    clean_corpus.append(clean_line.strip())
email_words=word_freqs.keys()            
print(len(email_words))

6976


## 以 regular expression 去除 email、數字、URL、Phone No、序號

In [111]:
re_pattern_list=[]
re_pattern_list.append('[a-zA-Z0-9_]+\.(com|org|net)[a-zA-Z0-9_\.]*$') #email
re_pattern_list.append('[0-9_]+') # all digits
re_pattern_list.append("\/\/[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&'\(\)\*\+,;=.]+$") # url
re_pattern_list.append('[0-9\-]+') # phone no


In [112]:
import re
keyword_list_new=[]
keyword_list = set(email_words)
for item in keyword_list:
    is_re_list = False
    for pattern1 in re_pattern_list:
        result=re.findall(pattern1, item)
        if  len(result) > 0:
            is_re_list = True
            break
    if is_re_list == False:
        keyword_list_new.append(item)
len(keyword_list_new)

4804

## 找出無母音的關鍵字

In [113]:
keyword_list_NoVowel={}
i=0
for word in keyword_list_new:
    if not set('aeiou').intersection(word) and len(word.replace('=','')) > 0 and len(word.replace('+','')) > 0 :
        if word in keyword_list_NoVowel.keys():
            keyword_list_NoVowel[word]+=1
        else:
            keyword_list_NoVowel[word]=1
#print(keyword_list_NoVowel.keys())   

## Save 無母音的關鍵字

In [114]:
import pickle
with open("keyword_list_NoVowel.pickle", 'wb') as f:
    pickle.dump(keyword_list_NoVowel, f)


## get List of Microsoft software

In [115]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

# keyword_list_TERM： {key:{converted_word, count}}
keyword_list_TERM={}
html_page = urlopen("https://en.wikipedia.org/wiki/List_of_Microsoft_software")
soup = BeautifulSoup(html_page)
div = soup.find('div', attrs={'id': 'mw-content-text'})
footer = str(div.contents).rfind("Misc.")

for link in div.findAll('a', attrs={'href': re.compile("/wiki/")}): #"^https://")}):
    link_text = link.text.replace('+','').lower().strip()
    # lemmatize
    link_text=stemmer.stem(link_text)
    #if link_text=='Microsoft Edge'.lower():
    #    print(stemmer.stem(link_text))
    if link_text in keyword_list_TERM.keys():
        keyword_list_TERM[link_text]['count']+=1
    else:
        keyword_list_TERM[link_text]={'converted_word':link_text.replace(" ", "_")}
        keyword_list_TERM[link_text]['count']=1

    link_text = link.text.replace('+','').lower().replace('microsoft', 'ms').strip()
    # lemmatize
    link_text=stemmer.stem(link_text)
    if link_text in keyword_list_TERM.keys():
        keyword_list_TERM[link_text]['count']+=1
    else:
        keyword_list_TERM[link_text]={'converted_word':link_text.replace(" ", "_")}
        keyword_list_TERM[link_text]['count']=1

    link_text = link.text.replace('+','').lower().replace('microsoft', '').strip()
    # lemmatize
    link_text=stemmer.stem(link_text)
    if link_text in keyword_list_TERM.keys():
        keyword_list_TERM[link_text]['count']+=1
    else:
        keyword_list_TERM[link_text]={'converted_word':link_text.replace(" ", "_")}
        keyword_list_TERM[link_text]['count']=1
        
    #print (link_text)
    if link_text == 'Windows To Go'.lower(): 
        break

link_text= 'window'       
if link_text in keyword_list_TERM.keys():
    keyword_list_TERM[link_text]['count']+=1
else:
    keyword_list_TERM[link_text]={'converted_word':link_text}
    keyword_list_TERM[link_text]['count']=1


In [116]:
key_list=list(keyword_list_TERM.keys())
for key1 in key_list:
    if key1.startswith('list of '):
        del keyword_list_TERM[key1]
#keyword_list_TERM.keys()

## get Ubuntu Glossaries

In [117]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

html_page = urlopen("https://help.ubuntu.com/community/Glossary")
soup = BeautifulSoup(html_page)
for link in soup.findAll('p', attrs={'class': re.compile("^line")}): #"^https://")}):
    strong_tag = link.find('strong')
    if not strong_tag == None:
        strong_tag_text = strong_tag.text.lower().strip()
        strong_tag_text=stemmer.stem(strong_tag_text)
        if strong_tag_text == 'Contributors:'.lower():
            break
        if strong_tag_text.find('(or') >= 0 and strong_tag_text.endswith(')'):
            #print('two_glossary={}'.format(strong_tag_text))
            two_glossary = strong_tag_text.split('(or')
            for item in two_glossary:
                item=item.strip().replace(")", '')
                item=stemmer.stem(item)
                if item in keyword_list_TERM.keys():
                    keyword_list_TERM[item]['count']+=1
                else:
                    keyword_list_TERM[item]={'converted_word':item.replace(" ", "_")}
                    keyword_list_TERM[item]['count']=1
        else:    
            if strong_tag_text.find('(also') >= 0 :
                strong_tag_text=strong_tag_text[:strong_tag_text.find('(also')]
            strong_tag_text=strong_tag_text
            if strong_tag_text.find('ubuntu') >= 0 :
                strong_tag_text='ubuntu'
            if strong_tag_text in keyword_list_TERM.keys():
                keyword_list_TERM[strong_tag_text]['count']+=1
            else:
                keyword_list_TERM[strong_tag_text]={'converted_word':strong_tag_text.replace(" ", "_")}
                keyword_list_TERM[strong_tag_text]['count']=1
        #print (strong_tag_text)


## Save Keywords

In [118]:
import pickle
with open("keyword_list.pickle", 'wb') as f:
    pickle.dump(keyword_list_TERM, f)
#keyword_list_TERM.keys()

## Save clean and original corpus(emails)

In [119]:
with open("clean_corpus.txt", 'w', encoding='utf8') as f:
    for line in clean_corpus:
        f.write(line.replace('\n', ' ')+'\n')
with open("original_corpus.txt", 'w', encoding='utf8') as f:
    for line in original_corpus:
        f.write(line.replace('\n', ' ')+'\n')
